### 场景

定义除法，根据参数，忽略ZeroDivisionError返回无穷大。OverflowError异常返回0.（越界）

In [3]:
def safe_division(number, divisor, ignore_overflow, ignore_zero_division):
    try:
        return number / divisor
    except OverflowError:
        if ignore_overflow:
            return 0
        else:
            raise
    except ZeroDivisionError:
        if ignore_zero_division:
            return float('inf')
        else:
            raise

float溢出返回0

In [4]:
result = safe_division(1.0, 10**500, True, False)
print(result)

0


0为除数，返回无穷大

In [5]:
result = safe_division(1, 0, False, True)
print(result)

inf


缺点：调用者写代码或后来的人阅读代码时，分不清参数的对应关系，导致难以排查bug。提升代码可读性的一种办法，是使用关键字参数。

In [11]:
def safe_division_b(number, divisor, ignore_overflow=False,ignore_zero_division=False):
    #...
    try:
        return number / divisor
    except OverflowError:
        if ignore_overflow:
            return 0
        else:
            raise
    except ZeroDivisionError:
        if ignore_zero_division:
            return float('inf')
        else:
            raise

现在调用者可以根据具体需要传入关键字参数。

In [13]:
safe_division_b(1, 10**500, ignore_overflow=False)
safe_division_b(1, 0, ignore_zero_division=True)

inf

**缺陷：** 调用者依然可以按照位置参数的形式来调用函数。我们没有办法确保调用者一定按照关键字的形式进行参数调用。

In [14]:
safe_division_b(1, 10**500, True, False)

0.0

### 强制以关键字方式调用 

In [15]:
def safe_division_c(number, divisor, *, 
                    ignore_overflow=False, 
                    ignore_zero_division=False):
    
    try:
        return number / divisor
    except OverflowError:
        if ignore_overflow:
            return 0
        else:
            raise
    except ZeroDivisionError:
        if ignore_zero_division:
            return float('inf')
        else:
            raise

\* 代表位置参数的结束，后面的参数在进行指定时，必需按照关键字的形式。

In [16]:
safe_division_c(1, 10**500, True, False)

TypeError: safe_division_c() takes 2 positional arguments but 4 were given

In [17]:
safe_division_c(1, 0, ignore_zero_division=True) #OK

try:
    safe_division_c(1,0)
except ZeroDivisionError:
    pass  #Expected

### Python2 的解决方案 

Python2中使用 ** \*\* **操作符, \*\* 操作符与 \* 操作符类似（第18条），但区别在于，它不是用来接受可变的位置参数，而是用来接受任意数量的关键字参数。即使某些关键字没有在函数中定义（使用），也接受。

\# Python 2
def print_args(*args, **kwargs):
    print('Positional: %s' % (args))
    print('Keyword: %s' % kwargs)
    
print_args(1, 2, foo='bar', stuff='meep')

函数接受 \*\*kwargs 参数，用pop方法把期望的关键字从kwargs字典了取走。如果取不到关键字，就以pop第二个参数为默认值。最后，为了防止调用者传入无效的参数，我们需要确定kwargs字典里已经没有关键字参数了。

\# Python2
def safe_divison_d(number, divisor, **kwargs):
    ignore_overflow = kwargs.pop('ignore_overflow', False)
    ignore_zero_div = kwargs.pop('ignore_zero_division', False)
    if kwargs:
        raise TypeError('Unexpected ** kwargs: %r' % kwargs)
    # ...

### 要点 

1. 函数调用使用关键字参数形式可以是代码更加明晰
2. 多参数容易混淆时，通过 '\*'强制后面的参数调用使用关键字参数形式。对应接受多个Boolean标志的函数，更应该这样做。
3. Python2 使用 \*\*kwargs 参数并手工抛出TypeError异常来实现相同的功能。